In [ ]:
import copernicusmarine
import xarray as xr
import numpy as np
from dask.distributed import Client
import pandas as pd
from pathlib import Path
import hvplot.xarray

In [ ]:
# copernicusmarine.login()

In [ ]:
date_start = pd.date_range("1998-01-01", "2020-08-01", freq="1MS")
date_end = pd.date_range("1998-02-01", "2020-09-01", freq="1MS")

export_filepath = Path("../data/0_raw/Papa_forcing.zarr")

coordinates = {"minimum_latitude": 46, "maximum_latitude": 58, "minimum_longitude": -158, "maximum_longitude": -127}

physic_dataset_name = "cmems_mod_glo_bgc_my_0.083deg-lmtl-Fphy_PT1D-i"
physic_variable = [
    "sea_water_potential_temperature_vertical_mean_over_pelagic_layer",
    "sea_water_pelagic_layer_bottom_depth",
]
bio_dataset_name = "cmems_mod_glo_bgc_my_0.083deg-lmtl_PT1D-i"
bio_variable = [
    "net_primary_productivity_of_biomass_expressed_as_carbon_in_sea_water",
    "mass_content_of_zooplankton_expressed_as_carbon_in_sea_water",
]

latitude = np.arange(46.5, 58.5, 1)
longitude = np.arange(-157.5, -126.5, 1)

In [ ]:
for start, end in zip(date_start, date_end):
    print(f"Processing {start.strftime('%Y-%m-%d')} to {end.strftime('%Y-%m-%d')}")
    if (not export_filepath.exists()) or (end not in xr.open_dataset(export_filepath, engine="zarr").time):
        coordinates.update({"start_datetime": start.strftime("%Y-%m-%d"), "end_datetime": end.strftime("%Y-%m-%d")})
        print(coordinates)

        physic = copernicusmarine.open_dataset(dataset_id=physic_dataset_name, **coordinates, variables=physic_variable)
        bio = copernicusmarine.open_dataset(dataset_id=bio_dataset_name, **coordinates, variables=bio_variable)

        physic = physic.interp(latitude=latitude, longitude=longitude, method="linear").load()
        bio = bio.interp(latitude=latitude, longitude=longitude, method="linear").load()

        results = xr.merge([physic, bio])

        if export_filepath.exists():
            results = xr.concat([xr.open_dataset(export_filepath, engine="zarr"), results], dim="time")
        results.to_zarr(export_filepath, mode="w")

# Remove duplicate in time axis and save outside of the github repo


In [ ]:
from dask.distributed import Client

In [ ]:
client = Client()

In [ ]:
data = xr.open_dataset(
    "../data/0_raw/Papa_forcing.zarr",
    engine="zarr",
    chunks="auto",
)
data

In [ ]:
grouped_data = data.groupby("time").mean(dim="time").load()
grouped_data

In [ ]:
grouped_data.to_zarr("../../../../../../Data/phd/FORCING/Papa_forcing.zarr", mode="w")

In [ ]:
xr.load_dataset("../../../../../../Data/phd/FORCING/Papa_forcing.zarr", engine="zarr")